In [1]:
import pandas as pd

key2name = pd.read_csv('data/key_english.csv')
key2name.drop(columns=['t', 'g'], inplace=True)

bible_df = pd.read_csv('data/t_asv.csv')
bible_df = bible_df[(bible_df['b'] >= 40) & (bible_df['b'] <= 43)] # Matthew, Mark, Luke, John
bible_df = bible_df.merge(key2name, on='b')

In [2]:
# Given a reference return the text
def get_verse_text(verse : str):
    reference = verse.split('-')
    book, start = reference[0].split(' ')
    start_chapter, start_verse = start.split(',')
    if len(reference) == 2:
        end_chapter, end_verse = reference[1].split(',')
    else:
        end_chapter, end_verse = start_chapter, start_verse
    
    start = int(f"{key2name[key2name['n'] == book]['b'].values[0]:02d}{int(start_chapter):03d}{int(start_verse):03d}")
    end   = int(f"{key2name[key2name['n'] == book]['b'].values[0]:02d}{int(end_chapter):03d}{int(end_verse):03d}")
    text = bible_df[(bible_df['id'] >= start) & (bible_df['id'] <= end)]['t'].str.cat(sep=' ')
    return text

In [15]:
# Discorso della montagna (Mt 5,1 - 7,29)
mountain_discourse = get_verse_text('Matthew 5,1-7,29')
print(mountain_discourse)

# Passione di Cristo secondo Giovanni (Gv 18,1 - 19,42)
passion = get_verse_text('John 18,1-19,42')
print(passion)

verses = [
    'Matthew 1,18-1,25',    # Nascità di Gesù secondo Matteo
    'Matthew 3,13-3,17',    # Battesimo di Gesù secondo Matteo
    'Matthew 4,1-4,11',     # Tentazioni di Gesù secondo Matteo
    'Matthew 5,1-7,29',     # Discorso della montagna
    'Matthew 9,18-9,26',    # Risurrezione della figlia di Giairo e guarigione della donna malata
    'Matthew 11,37-11,39'   # Rinnegarsi per seguire Gesù
    'Matthew 13,1-13,23',   # Parabola del seminatore
    'Matthew 13,44-13,46',  # Parabola del tesoro e della perla
    'Matthew 17,1-17,8',    # Trasfigurazione di Gesù
    'Matthew 18,1-18,5',    # I più piccoli nel regno dei cieli
    'Matthew 28,1-28,8'     # Risurrezione di Gesù secondo Matteo
    'Mark 1,9-1,11',        # Battesimo di Gesù secondo Marco
    'Mark 4,1-4,20',        # Parabola del seminatore
    'Mark 4,35-4,41',       # Tempesta sedata
    'Mark 8,34-8,38',       # Rinnegarsi per seguire Gesù
    'Mark 9,2-9,8',         # Trasfigurazione di Gesù
    'Mark 10,13-10,16',     # I più piccoli nel regno dei cieli
    'Mark 16,1-16,8'        # Risurrezione di Gesù secondo Marco
    'Luke 1,26-1,38',       # Annunciazione a Maria
    'Luke 2,1-2,20',        # Nascita di Gesù secondo Luca
    'Luke 3,21-3,22',       # Battesimo di Gesù secondo Luca
    'Luke 4,1-4,13',        # Tentazioni di Gesù secondo Luca
    'Luke 6,20-6,49',       # Discorso della montagna secondo Luca
    'Luke 7,1-7,10',        # Guarigione del servo del centurione
    'Luke 8,4-8,15',        # Parabola del seminatore
    'Luke 9,18-9,27',       # Trasfigurazione di Gesù
    'Luke 9,46-9,48',       # I più piccoli nel regno dei cieli
    'Luke 15,11-15,32',     # Parabola del figliol prodigo
    'Luke 24,1-24,12'       # Risurrezione di Gesù secondo Luca
    'John 1,1-1,18',        # Prologo del Vangelo secondo Giovanni
    'John 1,29-1,34',       # Battesimo di Gesù secondo Giovanni
    'John 2,1-2,11',        # Nozze di Cana
    'John 3,1-3,21',        # Incontro con Nicodemo
    'John 4,1-4,42',        # Incontro con la Samaritana
    'John 11,1-11,44',      # Risurrezione di Lazzaro
    'John 13,1-17,26',      # Ultima cena
    'John 20,1-20,18'       # Risurrezione di Gesù secondo Giovanni
]

And seeing the multitudes, he went up into the mountain: and when he had sat down, his disciples came unto him: and he opened his mouth and taught them, saying, Blessed are the poor in spirit: for theirs is the kingdom of heaven. Blessed are they that mourn: for they shall be comforted. Blessed are the meek: for they shall inherit the earth. Blessed are they that hunger and thirst after righteousness: for they shall be filled. Blessed are the merciful: for they shall obtain mercy. Blessed are the pure in heart: for they shall see God. Blessed are the peacemakers: for they shall be called sons of God. Blessed are they that have been persecuted for righteousness' sake: for theirs is the kingdom of heaven. Blessed are ye when `men' shall reproach you, and persecute you, and say all manner of evil against you falsely, for my sake. Rejoice, and be exceeding glad: for great is your reward in heaven: for so persecuted they the prophets that were before you. Ye are the salt of the earth: but i